# Stochastic Gradient Hamiltonian Monte Carlo

#### Kezi Cheng, Michael Lee, Daniel Olal, Victor Sheng
#### Harvard University, 29 Oxford Street, Cambridge MA 02138


### Abstract:
Hamiltonian Monte Carlo (HMC) and its variants are the state of the art for evaluating high dimensional Bayesian posteriors, which are often intractable by other methods.  A key step in the HMC sampler involves the evaluation of the gradient of the potential energy function for the entire dataset which is used for momentum update. Similarly, a Metropolis Hastings accept-reject must also be computed for the entire dataset. These two steps thus pose a limitation for applying HMC methods to big data such as streaming data. The authors propose a clever method called Stochastic Gradient Hamiltonian Monte Carlo (SGHMC) to surmount these two challenges. They first approximate the true gradient potential energy function using a noisy potential energy function estimate computed from a randomly sampled subset  of samples and then correct this estimate by adding friction thereby maintaining the desired distribution as the invariant distribution.  These approximations introduce additional hyper parameters into SGHMC that need to be tuned. In this project, we find that SGHMC is highly sensitive to hyper parameters for multi-dimensional distribtions for both Bayesian Linear and Logistic Regression. 

### Introduction

#### 1.1 Problem Statement and Context

Hamiltonian Monte Carlo (HMC) sampling methods define distant proposals with high acceptance probabilities in a Metropolis-Hastings framework where discretization of continuous-time system is needed. The drawback of simulating the Hamiltonian dynamical system is the need to compute the gradient of the potential energy function. This gradient computation becomes infeasible since they utilize the entire data set, which could be millions of observations. In the paper "Stochastic Gradient Hamiltonian Monte Carlo", Chen et al. combine the efficiencies in state space exploration of HMC with big data computational efficiencies of stochastic gradients, for large scale and online Bayesian sampling to rapidly explore the posterior (2014). This paper shows that SGHMC is important because it enables significant scaling of Bayesian methods in cases which would otherwise be computationally infeasible. 

#### 1.2 Background and Literature Review 

Recently, there has been a surge of gradient based approaches to utilize a noisy estimate of the gradient based on a mini-batch of data to scale algorithms. However, a majority of these use optimization approaches which brings to question whether sampling-based algorithms for Bayesian Inference could be of similar efficiency. There have been developments of HMC to be more applicable in a variety of settings. For example, Hoffman & Gelman (2011) combine the "No U-Turn" sampler with Wang (2013) to allow automatic tuning of the step size, $\epsilon$, and number of simulation steps, $m$. Welling & Teh (2011), Ahn et al (2012), and Patterson & Teh (2013) have proposed the Stochastic Gradient Langevin Dynamics (SGLD) in the sampling context which builds on the first-order Langevin dynamics without the momentum term of HMC. Stochastic Gradient Langevin Dynamics is an iterative optimization algorithm which introduces additional noise to the stochastic gradient descent estimator used to optimize a differentiable loss function.  

#### 1.3 Unique Contribution

Hamiltonian Monte Carlo provides a method for proposing samples from the posterior distribution of $\theta$ in a Metropolis-Hastings (MH) framework that efficiently explores the state space. The Hamiltonian function is defined by $H(\theta,r)=U(\theta)+\frac{1}{2}r^{T}M^{-1}r$ where $\theta$ represents position and $r$ represents momentum variables. To propose samples, HMC simulates $d\theta=M^{-1}rdt$ and $dr=-\nabla U(\theta)dt$. In practice, we simulate a discretized system using the "leapfrog" method which introduces inaccuracies and the need for a MH step. 

If a stochastic gradient modification was directly applied to HMC to assess the impact of a noisy gradient, the stochastic gradient would no longer lead to Hamiltonian dynamics with the desired target distribution as the stationary distribution. This implies that an injected noise can be corrected through an MH step, but this would require costly computations on the entire data set. In addition, carrying out long simulation runs before an MH step could lead to low acceptance rates due to large deviations in the Hamiltonian from the injected noise.

This paper introduces a variant that uses second-order Langevin dynamics (Wang and Uhlenbeck, 1945) with a friction term that counteracts the effects of the noisy gradient estimate. The dynamics were observed assuming the injected noise is Gaussian. For discretized systems, the authors allow step size go to zero so the MH step is not needed, which leads to significant computational gain. 

#### 1.4 High Level Technical Content

This paper proposes incorporating friction into a naive implementation of stochastic gradient HMC.The paper gives the analogy of naive stochastic HMC as playing ice hockey with random wind blowing, serving as the noise. It then uses the street hockey analogy to demonstrate the addition of a friction term. The asphalt provides friction such that even if the wind is blowing randomly, the surface prevents the puck from running too far away. It then applies the SGHMC algorithm to a classification task using Bayesian neural networks and to online Bayesian matrix factorization of a standard movie dataset to demonstrate the effectiveness and compuational gain as compared to other implementations of HMC. 

#### 1.5 Detailed Technical Contribution

To eliminate the need for a costly MH correction step while preserving Hamiltonian dynamics by minimizing the effect of the injected noise, this paper proposes a friction term, $BM^{-1}r$, to the momentum update that helps to decrease the energy $H(\theta,r)$ and reduce the influence of noise.  

**Theorem 3.2** states $\pi(\theta,r)\propto exp(-H(\theta,r))$ is the unique stationary distribution of the dynamics described by:<br/>

$d\theta = M^{-1}rdt$ <br/>
$dr=-\nabla U(\theta)dt-BM^{-1}rdt+\mathcal{N}(0,2Bdt)$<br/>

This is significant because unlike the naive stochastic gradient HMC where the distribution $\pi(\theta,r)$ makes the dynamics no longer invariant, the added friction term in addition to the noise using second-order Langevin dynamics shows similar invariance property to the original Hamiltonian dynamics. 

An important factor in which the "friction" term is decided upon, is the assumption that we know the noise model $B$. To implement SGHMC in practice requires a user specified friction term $C \geq\widehat{B}$ where $\widehat{B}$ is an estimate of the noise. **Proposition 3.1** states that in an unrealistic scenario of the perfect estimation of $B$ where $\widehat{B}=B$, then the dynamics of $dr=-\nabla U(\theta)dt-CM^{-1}rdt+\mathcal{N}(0,2(C-\widehat{B})dt)+\mathcal{N}(0,2Bdt)$ <br/> yield a stationary distribution $\pi(\theta,r)\propto e^{-H(\theta,r)}$. In this case, the momentum update simplifies to $r=-\nabla U(\theta)dt-CM^{-1}rdt+\mathcal{N}(0,2Cdt)$ <br/> where both the friction and noise term are replaced with $C$ in place of $B$. 

In practice, we assume inaccurate estimation of $B$, which makes the choice of $M,C,\widehat{B}$, and the complexity in estimating $\nabla\widehat{U}(\theta)$ extremely important. If we allow $\widehat{B}$ to be an arbitrary $d x d$ positive definite matrix, the time complexity for the $(\theta,r)$ update is $O(d^{3})$. In a more realistic yet simple case where we have an inaccurate estimation of the true noise model, $\widehat{B}=0$, then the combined $2Cdt$ and $-CM^{-1}rdt$ dominates over the true noise which goes to 0 as step size $\epsilon$ goes to 0.  

The **SGHMC algorithm** is as follows: 

**for** $t=1,2...$ **do** <br/>
*optionally, resample momentum r as* <br/>
    *$r^{t}\sim\mathcal{N}(0,M)$* <br/>
    $(\theta_0, r_0)=(\theta^{(t)},r^{(t)})$ <br/>
    *simulate dynamics:* <br/>
    **for** $i=1$ to m **do** <br/>
        $\theta_i\leftarrow\theta_{i-1}+\epsilon_{t}M^{-1}r_{i-1}$ <br/>
            $r_i\leftarrow r_{i-1}+\epsilon_{t}\nabla\widetilde{U}(\theta_i)-\epsilon_{t}CM^{-1}r_{i-1}+\mathcal{N}(0,2(C-\widehat{B})\epsilon_t)$ <br/>
    **end** <br/>
    ($\theta^{(t+1)},r^{(t+1)})=(\theta_m,r_m)$, no M-H step <br/>
    **end**
           
The main difference between SGHMC and HMC is the friction and noise term in the momentum update, as well as the lack of Metropolis Hasting correction step in the SGHMC.




### Simulated Experiments

#### 2.1 Experiments

The paper empirically explores the behavior of HMC using exact gradients relative to stochastic gradients. Figure 1 demonstrates the simulated experiments where momentum is resampled every 50 steps to validate theoretical results and demonstrate the differences between i) exact HMC, ii) naive implementation of stochastic gradient HMC, iii) incorporating friction into SGHMC, with and without MH steps. In the adjacent figure, we implemented the author's code to validate their findings. In their findings, they observe that even without an MH correction step, both the HMC and SGHMC algorithms provide results close to the true distribution whereas results of the naive stochastic gradient HMC diverge significantly from the truth distribution unless the MH step is added. In addition, discretized Hamiltoninan dynamical systems associated with the various samplers were compared and the results match those of Figure 1 where noisy system without friction diverges significantly. They note that the behavior can fully be corrected through adding friction, and although periodic resampling of the momentum helps control divergence, the HMC stationary distribution is not correct (Figure 2).  

Efficiency in sampling is a key driver for using HMC over other MCMC algorithms. This paper calculates the autocorrelation time of ten million samples and the average absolute error of the resulting sample covariance for SGLD and SGHMC. Figure 3 shows SGHMC is efficient at exploring the distribution as it achieves lower estimation error at low autocorrelation times, and momentum variable associated with SGHMC drives the sampler to move along distribution contours.

In addition, the paper tests the different methods on a handwritten digits classification task using the MNIST dataset, containing 60,000 training and 10,000 testing instances. A two layer Bayesian neural network with 100 hidden variables using a sigmoid unit and an output layer using softmax is used for classification. Figure 4 shows that SGD with momentum converges faster than SGD. SGHMC has an advantage over SGLD as well as optimization methods as it converges to a low test error much more rapidly. This result is further validated with lower RMSE values estimated using 5-fold cross validation for inferring parameters of a Bayesian probabilistic matrix factorization model for sampling rather than optimization methods. 

**Figure 1**<br/>
<img src="data/paper_figure1.png" width="350" align="left"/>
<img src="data/paper_figure1a.png" width="400" align="center"/>

**Figure 2**<br/>
<img src="data/paper_figure2.png" width="300" align="left"/>

**Figure 3**<br/>
<img src="data/paper_figure3.png" width="500" align="left"/>

**Figure 4**<br/>
<img src="data/paper_figure4.png" width="300" align="left"/>


#### 2.2 Evaluation

The authors provide a compelling argument for using a friction term which offsets the need to use MH correction.  While their experiments validate the hypothesis set forth that SGHMC is an efficient method, the authors do not clearly specify how they picked the optimal hyperparameters for the model. It is clear that hyper parameters play an important role in generating the right posterior distribution and therefore their model only works well if you estimate the hyperparameters correctly or if you shrink the step size towards 0, which is infeasible.       

#### 2.3 Future Work

In the case that we don’t obtain the correct friction term to offset the noise, SGHMC performs poorly. This could be improved by adding an MH step, but this circles back to the original problem that the SGHMC algorithm tries to address, which is to reduce computational cost without having to sample all of the data points. However, Korattikara et al. demonstrate a method in "Cutting the Metropolis-Hastings Budget" that can cut back the computational cost by dialing down bias at a rate that optimally balances error through an 𝜺 knob (2014). They conducted sequential hypothesis tests to decide whether to accept or reject a given sample and find that the majority of these decisions can be made based on a small fraction of the data with high confidence, by using a minibatch of size m for the first test and increasing m when required. It is possible to consider combining Korattikara's work with SGHMC to use an efficient MH correction term to improve the quality of sampling and minimizing computational cost. 

### Pedagogical Examples

#### 3.1 Bayesian Linear Regression

To examine the behavior of the SGHMC sampler, we first implemented a bivariate Bayesian linear regression example with normal priors on the coefficients, since in this case we know the true form of the posterior is bivariate normal.

We constructed a simulated dataset of 1000 data points using sklearn's make_regression() function, seen below:

![](data/regression_data.png)

As a unit test of our SGHMC implementation, we first confirmed that we can correctly sample from an arbitrary bivariate normal distribution:

![](data/mvn_unit_test.png)


The SGHMC algorithm relies on the assumption that the gradient noise is normally distributed, which the authors argue should be true via the Central Limit Theorem given a batch size in the hundreds. We assessed this claim by computing the gradient of the potential energy function (negative log pdf of a bivariate normal) at the point [20.3, -1.8] using a batch size of 150 (out of 1000 total data points). The distribution does indeed look normal, though not necessarily centered at 0 like the paper suggests:

![](data/gradient_noise.png)


The issue with trying to estimate the gradient noise empirically like this is that the distribution of the noise depends on the location at which we are evaluating the gradient, whereas the algorithm only accepts a single estimated noise covariance matrix.



##### 3.1.1 Trying Different Batch Sizes

![](data/vary_batch_sizes.png)

With the true posterior as a benchmark, we now try to tweak the algorithm parameters to produce the best quality samples. First, we assess the effect of batch size on the quality of posterior samples, keeping other parameters fixed. When we use the full dataset (batch size = 1000), our samples match the true posterior very well. However, as we decrease the batch size, both bias and variance steadily increase in the samples.

##### 3.1.2 Trying Different Friction Settings

![](data/vary_friction.png)

Second, we evaluate different values of friction, keeping other parameters fixed. There seems to be a sweet spot for friction: no friction results in inflated variance, while too much friction constrains the simulated trajectories too much. In our case, a setting of 10 seems ideal.

##### 3.1.3 Trying Different Noise Estimates

![](data/vary_noise.png)

Third, we see if attempting to estimate the gradient noise makes a difference. The authors recommend keeping the noise estimate at 0 and using a small step size since it's difficult to correctly estimate the noise, but we will try. In this case, the noise estimates don't see to make much of a difference.

##### 3.1.4 Trying Different Step Sizes

![](data/vary_step.png)

Finally, we assess the effect of step size on the quality of the posterior samples. Step size is critically important to tune properly because it serves a dual purpose: controlling the size of each leapfrog step and also shrinking the actual gradient noise. If the step size is too small, our proposals via simulated dynamics don't travel very far and thus inefficiently explore the state space. However, as we increase the step size, the effect of the true gradient noise grows, which is problematic. We see this trade-off play out in the cases illustrated above: in our case, a step size of 0.001 seems optimal.

##### 3.1.5 Posterior Predictive Check

While we can clearly see that our best effort posterior samples are still slightly biased (though variance is comparable to the true distribution), it's important to see how this actually translates to the resulting regression lines derived from the sampled coefficients. In this case, using the biased samples result in regression lines that are practically indistinguishable from the ones generated from the true posterior samples:

![](data/posterior_predictive.png)

Of course, in many settings we will not know the form of the true distribution. Next, we will explore a more complex example involving logistic regression.

#### 3.2 Bayesian Logistic Regression

Our second pedadogical example explores the use of Stochastic Gradient HMC In Bayesian Logistic Regression.  We start by generating an articial dataset using sklearn's make_classification() function.  

Bayesian Logistic Regression is routinely used in text categorization, gene expression analyses and longitudinal clinical trials. Maximum likelihood and standard logistic regression approaches perform poorly in such cases due to lack a convergence as a result of numerical ill conditioning. On the contrary, Bayesian Logistic regression avoids overfitting by using priors (Genkin et al., 2007).  

We studied the effects of changing the friction term, noise model, batch size, step size in SGHMC. We used the accuracy scores and trace plots as metrics for evaluating performance on dataset comprising of two features and 2000 observations. We used a standard HMC implementation of the same dataset for comparison.

![HMC](data/HMC.png)

##### 3.2.1 Effect of Changing the Friction Term (C)

![friction_c](data/friction_c.png)

For evaluating the effect of the friction term, C, on the SGHMC performance, we fixed the step size, the noise model, batch size and iterated over different values of C. We find that smaller values C perform better with higher accuracy scores. However, a visual inspection of the decision boundary shows high bias and high variance.

##### 3.2.2 Effect of Changing the Noise Term (B)

We next fixed the Friction term C at 1 (as suggested by the authors), stepsize, batch size and varied the noise term B. We tested B at the following levels: 0, 0.1, 1 and 10. We did not see a large difference in logistic regression classification accuracy. The uncertainty in the parameter estimates appears to be similiar across all the values of B that we tested. The results also suggest that B and C may counteract or mask each other’s effect. It would therefore make sense to do a grid search for these two simultaneously and evaluate the performance of the algorithm. 

![noise_b](data/noise_b.png)

We do not see a large difference in logistic regression classification accuracy when we adjust B, the noise term. The uncertainty in the parameter estimates appears to be similiar across all the values of B that we tested.  

##### 3.2.3 Effect of Changing the Batch Size 

![batchsize](data/batchsize.png)

The logistic regression model accuracy appears to be the highest with a batch size of 500.  The uncertainty in our parameter estimates increases when we decrease the batch size.  

##### 3.2.4 Effect of Changing the Step Size

![stepsize](data/stepsize.png)

As expected, the step size has a significant effect on the accuracy of SGHMC sampler. As we increase the stepsize greater than 0.005, the classification accuracy decreases and the level of uncertainty in the parameter estimates increases significantly.  

### Conclusion

The Stochastic Gradient Hamiltonian Monte Carlo method which Chen et al introduce in their paper demonstrates how a stochastic gradient implementation of the Hamiltonian Monte Carlo sampler can save on computational cost, which could be useful in big data applications.  However, we found that the hyper parameters which the authors used were highly tailored to their specific application and the resulting posterior distribution from a SGHMC sampler is highly sensitive to the choice of step size, friction, and batch size parameters.  Therefore, the SGHMC sampler should not be used unless the hyper parameters are optimized using a gridsearch; there does not appear to be a standard rule of thumb that can be used to tune the hyper parameters.  

### References

Chen, Fox, Guestrin.  Stochastic Gradient Hamiltonian Monte Carlo. 2014  https://arxiv.org/abs/1402.4102  

Korattikara, Chen, Welling.  Austerity in MCMC Land: Cutting the Metropolis-Hastings Budget, 2013.  https://arxiv.org/abs/1304.5299